# Tensorflow 2 构建人工神经网络

## 低阶 API

In [1]:
# %% 导入依赖项
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

C:\Users\Dragon1573\Repos\tensorflow-abc\env\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
  return f(*args, **kwds)


In [2]:
digits = load_digits()
# 独热编码
digits_y = np.eye(10)[digits.target.reshape(-1)]
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits_y, test_size=0.2, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1437, 64), (360, 64), (1437, 10), (360, 10))

包含单个隐含层，输入层长度`(None, 64)`，隐含层`(None, 30)`，输出层`(None, 10)`，隐含层 RELU 激活，输出层不激活。

In [3]:
class Model(object):
    """ 模型类 """
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([64, 30]))
        self.b1 = tf.Variable(tf.random.normal([30]))
        self.W2 = tf.Variable(tf.random.normal([30, 10]))
        self.b2 = tf.Variable(tf.random.normal([10]))

    def __call__(self, x):
        x = tf.cast(x, tf.float32)
        # 线性计算＋激活
        fc1 = tf.nn.relu(tf.add(tf.matmul(x, self.W1), self.b1))
        fc2 = tf.add(tf.matmul(fc1, self.W2), self.b2)
        return fc2

In [4]:
def loss_fn(model, x, y):
    """ 损失函数 """
    preds = model(x)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y))

In [5]:
def accuracy_fn(logits, labels):
    """ 准确率评估函数 """
    preds = tf.argmax(logits, axis=1)
    labels = tf.argmax(labels, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [6]:
# %% 迭代训练
EPOCHS, LEARNING_RATE = 200, 0.02
model = Model()
for epoch in range(EPOCHS):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, X_train, y_train)
    trainable_variables = [model.W1, model.b1, model.W2, model.b2]
    grads = tape.gradient(loss, trainable_variables)
    # Adam 是一种优化的梯度下降
    optimizer = tf.optimizers.Adam(learning_rate=LEARNING_RATE)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    accuracy = accuracy_fn(model(X_test), y_test)
    # 输出指标
    if (epoch + 1) % 10 == 0:
        print('Epoch [{}/{}], Train loss: [{:.3f}], Test accuracy: {:.3f}'
              .format(epoch + 1, EPOCHS, loss, accuracy))

Epoch [10/200], Train loss: [65.105], Test accuracy: 0.389
Epoch [20/200], Train loss: [23.203], Test accuracy: 0.625
Epoch [30/200], Train loss: [15.269], Test accuracy: 0.689
Epoch [40/200], Train loss: [11.559], Test accuracy: 0.747
Epoch [50/200], Train loss: [8.845], Test accuracy: 0.786
Epoch [60/200], Train loss: [7.094], Test accuracy: 0.822
Epoch [70/200], Train loss: [6.263], Test accuracy: 0.836
Epoch [80/200], Train loss: [5.473], Test accuracy: 0.853
Epoch [90/200], Train loss: [4.588], Test accuracy: 0.856
Epoch [100/200], Train loss: [3.989], Test accuracy: 0.878
Epoch [110/200], Train loss: [3.532], Test accuracy: 0.872
Epoch [120/200], Train loss: [3.029], Test accuracy: 0.872
Epoch [130/200], Train loss: [2.855], Test accuracy: 0.883
Epoch [140/200], Train loss: [2.724], Test accuracy: 0.894
Epoch [150/200], Train loss: [2.248], Test accuracy: 0.892
Epoch [160/200], Train loss: [2.272], Test accuracy: 0.894
Epoch [170/200], Train loss: [2.164], Test accuracy: 0.900
Ep

## Keras 高阶 API 实现

### 函数式模型

In [8]:
# %% 构建模型
inputs = tf.keras.Input(shape=(64,))
x = tf.keras.layers.Dense(30, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

In [9]:
# %% 指定模型的输入与输出
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1950      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
Total params: 2,260
Trainable params: 2,260
Non-trainable params: 0
_________________________________________________________________


使用 Adam 优化器，Categorical Cross Entropy 多分类交叉熵作为损失函数。CCE 来自 Keras ，去除了 Softmax 步骤，此步骤被前移到模型输出激活层。

In [10]:
# %% 构建、训练与评估
model.compile(optimizer=tf.optimizers.Adam(), metrics=['accuracy'],
              loss=tf.losses.categorical_crossentropy)
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
23/23 [==============================] - 0s 10ms/step - loss: 7.7550 - accuracy: 0.1030 - val_loss: 4.6756 - val_accuracy: 0.1806
Epoch 2/10
23/23 [==============================] - 0s 3ms/step - loss: 3.7980 - accuracy: 0.2241 - val_loss: 2.9313 - val_accuracy: 0.3111
Epoch 3/10
23/23 [==============================] - 0s 2ms/step - loss: 2.5837 - accuracy: 0.3326 - val_loss: 2.0953 - val_accuracy: 0.4028
Epoch 4/10
23/23 [==============================] - 0s 3ms/step - loss: 1.9256 - accuracy: 0.4433 - val_loss: 1.5561 - val_accuracy: 0.5222
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 1.4264 - accuracy: 0.5484 - val_loss: 1.1886 - val_accuracy: 0.6333
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 1.0709 - accuracy: 0.6688 - val_loss: 0.8805 - val_accuracy: 0.7306
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 0.8310 - accuracy: 0.7502 - val_loss: 0.7066 - val_accuracy: 0.7917
Epoch 8/10
23/23 [=

### 层序模型

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=30, input_dim=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 30)                1950      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                310       
Total params: 2,260
Trainable params: 2,260
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
23/23 [==============================] - 0s 9ms/step - loss: 6.3734 - accuracy: 0.1670 - val_loss: 3.7045 - val_accuracy: 0.2639
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 2.7745 - accuracy: 0.3389 - val_loss: 1.7701 - val_accuracy: 0.4639
Epoch 3/10
23/23 [==============================] - 0s 3ms/step - loss: 1.5441 - accuracy: 0.5609 - val_loss: 1.1329 - val_accuracy: 0.6611
Epoch 4/10
23/23 [==============================] - 0s 3ms/step - loss: 1.0194 - accuracy: 0.7022 - val_loss: 0.8247 - val_accuracy: 0.7361
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 0.7610 - accuracy: 0.7752 - val_loss: 0.6606 - val_accuracy: 0.7889
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 0.6122 - accuracy: 0.8149 - val_loss: 0.5605 - val_accuracy: 0.8139
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5004 - accuracy: 0.8483 - val_loss: 0.4874 - val_accuracy: 0.8361
Epoch 8/10
23/23 [==

### 高自由度模型

In [14]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        # 模型层次初始化
        self.dense_1 = tf.keras.layers.Dense(30, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        """ 前向传播 """
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [15]:
model = Model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
23/23 [==============================] - 0s 8ms/step - loss: 6.8500 - accuracy: 0.1023 - val_loss: 4.2757 - val_accuracy: 0.2333
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 3.2667 - accuracy: 0.2867 - val_loss: 2.3362 - val_accuracy: 0.4056
Epoch 3/10
23/23 [==============================] - 0s 3ms/step - loss: 1.9067 - accuracy: 0.4690 - val_loss: 1.3475 - val_accuracy: 0.5806
Epoch 4/10
23/23 [==============================] - 0s 3ms/step - loss: 1.2023 - accuracy: 0.6207 - val_loss: 0.8918 - val_accuracy: 0.7222
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 0.8515 - accuracy: 0.7286 - val_loss: 0.6496 - val_accuracy: 0.8111
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 0.6506 - accuracy: 0.7871 - val_loss: 0.5231 - val_accuracy: 0.8389
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5231 - accuracy: 0.8309 - val_loss: 0.4421 - val_accuracy: 0.8750
Epoch 8/10
23/23 [==